In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from datetime import datetime
import json
import numpy as np
import os
import pandas as pd
import sys
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.callbacks import History
history = History()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Specify here the NN topology and the dataframe to be loaded

In [ ]:
#specify the NN topology here:
#input_layer_nodes = number_of_attributes
number_of_attributes = 27
hidden_layer_nodes = 27
output_layer_nodes = 4
BATCH_SIZE = 10
EPOCHS = 10
#specify the dataset here:
#choose the dataframe which the NN will be trained and mo
#df_number = 1
#number_of_samples = 2000

attributes_array = ['82', '83', '81', '89', '15', '22', '84', '90', '88', '79', '87', '80', '17', '64', '60', '10', '29', '59', '63', '24', '62', '61', '91', '48', '47', '32', '34', '31', '33', '45', '46', '36', '37', '38', '92']

###Define the model

In [ ]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'),
    tf.keras.layers.Dense(output_layer_nodes)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

###Import the data

In [ ]:
title_train = f'/content/drive/MyDrive/network-traffic-classification-main/csv-files/df_train_6000_samples_4_classes.csv'
title_test =  f'/content/drive/MyDrive/network-traffic-classification-main/csv-files/df_test_2000_samples_4_classes.csv'
df_train_raw = pd.read_csv(title_train)
df_test_raw = pd.read_csv(title_test)
target_train = df_train_raw.pop('target')
target_test = df_test_raw.pop('target')

# Select a sub-dataframe from the main dataframe with a lower number of attributes
df_train = df_train_raw[attributes_array[0:number_of_attributes]]
df_test = df_test_raw[attributes_array[0:number_of_attributes]]

df_train = np.asarray(df_train).astype(np.float32)
target_train = np.asarray(target_train).astype(np.float32)
df_test_np = np.asarray(df_test).astype(np.float32)
target_test_np = np.asarray(target_test).astype(np.float32)

In [ ]:
  # [OLD] -> in this 8x8x4 neural network, we will be working with just the 8 classes with most samples
  # therefore we already dropped drop classes 4, 8, 10 and 11
  # because of this, need to rename the remaining classes so we keep a list from 0 to 7.
  # class 9 (SERVICES) must become class 4.

# for i,value in enumerate(target_train):
#     if value == 9:
#       target_train[i] = 4

# for i,value in enumerate(target_test_np):
#     if value == 9:
#       target_test_np[i] = 4

for i,value in enumerate(target_train):
    if value == 5:
      target_train[i] = 2
    if value == 7:
      target_train[i] = 3

for i,value in enumerate(target_test_np):
    if value == 5:
      target_test_np[i] = 2
    if value == 7:
      target_test_np[i] = 3

### NN training
 model weights file generation

 model parameters json file generation

In [ ]:
# Begin NN training
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(df_train)

# Execute Trainning and Testing
print('Starting training for',number_of_attributes,'attributes')
model = get_basic_model()
model.fit(df_train, target_train, epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE)

test_loss, one_test_acc =  model.evaluate(df_test_np,  target_test_np, verbose=1, batch_size=BATCH_SIZE)
tf_predictions_probabilities = model.predict(df_test_np)
tf_predictions = []

for i,x in enumerate(tf_predictions_probabilities):
  #print("i:",i,"x_max:",x.max(),"x:",x)
  j_max = x.argmax()
  tf_predictions.append(j_max)

conf_m = tf.math.confusion_matrix(target_test_np,tf_predictions)
print(conf_m)

# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y %H:%M:%S")
# Report file content
report = {
    "datetime": dt_string,
    "attributes": attributes_array[0:number_of_attributes],
    "number_of_attributes": number_of_attributes,
    "hidden_layer_nodes": hidden_layer_nodes,
    "output_layer_nodes": output_layer_nodes,
    "accuracy_test": one_test_acc,
    "attributes_array": attributes_array,
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS
    #"number_of_samples": number_of_samples,
    #"df_number": df_number
}


# norm_layer_weights = model.layers[0].get_weights()[0] #weight
# norm_layer_biases  = model.layers[0].get_weights()[1] #bias
# hidden_layer_weights = model.layers[1].get_weights()[0] #weight
# hidden_layer_biases  = model.layers[1].get_weights()[1] #bias
# out_layer_weights = model.layers[2].get_weights()[0] #weight
# out_layer_biases  = model.layers[2].get_weights()[1] #bias
# print('\nInput -> Normalization Layer bias:\n',norm_layer_biases)
# print('\nInput -> Normalization Layer weights:\n',norm_layer_weights)
# print('\nNormalization -> Hidden Layer bias:\n',hidden_layer_biases)
# print('\nNormalization -> Hidden Layer weights:\n',hidden_layer_weights)
# print('\nHidden -> Output Layer bias:\n',out_layer_biases)
# print('\nHidden -> Output Layer weights:\n',out_layer_weights,'\n')



Starting training for 27 attributes
Epoch 1/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.4954 - accuracy: 0.8302
Epoch 2/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.2780 - accuracy: 0.9065
Epoch 3/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.2280 - accuracy: 0.9242
Epoch 4/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.1999 - accuracy: 0.9311
Epoch 5/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.1803 - accuracy: 0.9366
Epoch 6/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.1675 - accuracy: 0.9414
Epoch 7/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.1561 - accuracy: 0.9463
Epoch 8/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.1474 - accuracy: 0.9506
Epoch 9/10
2400/2400 [==============================] - 6s 2ms/step - loss: 0.1382 - accuracy: 0.9542
Epoch 10/10
250/250 [=========================

In [ ]:
# Writes into the file
# model parameters json file generation
title = f"/content/drive/MyDrive/network-traffic-classification-main/nn-reports/nn_model_parameters_{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}_{dt_string}.json"
with open(title, "w") as f:
  json.dump(report, f)

title_parameters_save = f"/content/drive/MyDrive/network-traffic-classification-main/nn-reports/nn_model_parameters_{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
# model weights file generation
title_model_save = f'/content/drive/MyDrive/network-traffic-classification-main/tf-models/nn_trained_model_weights_{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}_{dt_string}.csv'
model.save(title_model_save)

title_model_save = f'/content/drive/MyDrive/network-traffic-classification-main/tf-models/nn_trained_model_weights_{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.csv'
model.save(title_model_save)
